# Create label from mask image

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import measure
from skimage.measure import regionprops_table

for spe in ['Falciparum', 'Malariae','Ovale','Vivax']:
    path='./Data/four_species/'+ spe +'/gt/'
    gts=[file for file in os.listdir(path)]
    spe_gt=[]
    for gt in gts:
        image = Image.open(path + gt)
        spe_gt.append(np.array(image))
    label_path='./Data/four_species/'+spe+'/label/'
    os.mkdir(label_path)
    width_img=2592
    height_img=1944
    for i in range(len(spe_gt)):
        labeled_image= measure.label(spe_gt[i])
        name_image=gts[i][:-4]
        props = regionprops_table(labeled_image, properties=('centroid','bbox'))
        df_box=pd.DataFrame(props)
        
        file_path=label_path+name_image+'.txt'
        with open(file_path, 'w') as f:
            for j in range(df_box.shape[0]):
                if spe == 'Falciparum':
                    label=1
                elif spe =='Malariae':
                    label=2
                elif spe == 'Ovale':
                    label=3
                else:
                    label=4
                x_yolo=(df_box['bbox-1'][j]+df_box['bbox-3'][j])/2/width_img
                
                y_yolo=(df_box['bbox-0'][j]+df_box['bbox-2'][j])/2/height_img
                w_yolo=(df_box['bbox-3'][j]-df_box['bbox-1'][j])/width_img
                h_yolo=(df_box['bbox-2'][j]-df_box['bbox-0'][j])/height_img
                line = "{0} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n".format(label, x_yolo, y_yolo, w_yolo, h_yolo)
                
                f.write(line)